In [2]:
!pip install qiskit_aer #==0.3.2
!pip install numpy #==1.16.2
!pip install qiskit #==0.14.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 7.1 MB/s eta 0:00:00


In [4]:
from __future__ import unicode_literals
from math import sqrt
import random
from random import randint as rand

# Función para calcular el máximo común divisor utilizando el algoritmo de Euclides
def mcd(a, b):
    if b == 0:
        return a
    else:
        return mcd(b, a % b)

# Función para encontrar el inverso modular de 'a' bajo el módulo 'm'
def inverso_modular(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return -1

# Función para comprobar si un número 'n' es primo
def es_primo(n):
    if n < 2:
        return False
    elif n == 2:
        return True
    else:
        for i in range(1, int(sqrt(n)) + 1):
            if n % i == 0:
                return False
    return True

# Función para generar un par de claves RSA
def generar_par_claves(tamaño_clave):
    p = rand(1, 1000)
    q = rand(1, 1000)
    nMin = 1 << (tamaño_clave - 1)
    nMax = (1 << tamaño_clave) - 1
    primos = [2]
    inicio = 1 << (tamaño_clave // 2 - 1)
    fin = 1 << (tamaño_clave // 2 + 1)
    if inicio >= fin:
        return []
    for i in range(3, fin + 1, 2):
        for p in primos:
            if i % p == 0:
                break
        else:
            primos.append(i)
    while (primos and primos[0] < inicio):
        del primos[0]
    while primos:
        p = random.choice(primos)
        primos.remove(p)
        valores_q = [q for q in primos if nMin <= p * q <= nMax]
        if valores_q:
            q = random.choice(valores_q)
            break
    print(p, q)
    n = p * q
    phi = (p - 1) * (q - 1)
    e = random.randrange(1, phi)
    g = mcd(e, phi)
    while True:
        e = random.randrange(1, phi)
        g = mcd(e, phi)
        d = inverso_modular(e, phi)
        if g == 1 and e != d:
            break

    return ((e, n), (d, n))

# Función para encriptar un mensaje de texto plano utilizando la clave pública
def encriptar(msg_texto_plano, paquete):
    e, n = paquete
    msg_texto_cifrado = [pow(ord(c), e, n) for c in msg_texto_plano]
    return ''.join(map(lambda x: str(x), msg_texto_cifrado)), msg_texto_cifrado

# Función para desencriptar un mensaje cifrado utilizando la clave privada
def desencriptar(msg_texto_cifrado, paquete):
    d, n = paquete
    msg_texto_plano = [chr(pow(c, d, n)) for c in msg_texto_cifrado]
    return (''.join(msg_texto_plano))

In [5]:
# Se solicita al usuario ingresar el tamaño en bits de la clave
tamanio_bits = int(input("Ingrese el tamaño en bits: "))

# Se generan las claves pública y privada utilizando la función generar_par_claves
publica, privada = generar_par_claves(2 ** tamanio_bits)

Ingrese el tamaño en bits: 5
62497 41879


## Cifrado RSA

In [6]:
# Se solicita al usuario que escriba un mensaje
mensaje = input("\nEscribe el mensaje: ")

# Se encripta el mensaje utilizando la función encriptar y la clave pública
mensaje_encriptado, objeto_encriptado = encriptar(mensaje, publica)

# Se imprime el mensaje encriptado
print("\nMensaje encriptado: " + mensaje_encriptado)


Escribe el mensaje: fafas

Mensaje encriptado: 127802687997752143412780268799775214341532863624


## Descifrado RSA

In [7]:
# Se desencripta el mensaje utilizando la función desencriptar y la clave privada
mensaje_desencriptado = desencriptar(objeto_encriptado, privada)

# Se imprime el mensaje desencriptado
print("\nMensaje desencriptado usando el algoritmo RSA: " + mensaje_desencriptado)


Mensaje desencriptado usando el algoritmo RSA: fafas


## Algoritmo cuántico de Shor

In [8]:
from math import gcd,log
from random import randint
import numpy as np
from qiskit import *
from qiskit_aer import *

# Se obtiene el backend del simulador qasm de Qiskit Aer
simulador_qasm = Aer.get_backend('qasm_simulator')

In [9]:
# Función para encontrar el período de un número 'a' módulo 'N'
def periodo(a, N):

    # Número de qubits disponibles
    qubits_disponibles = 16
    r = -1

    # Verificación si N es demasiado grande para IBMQX
    if N >= 2 ** qubits_disponibles:
        print(str(N) + ' es demasiado grande para IBMQX')

    # Inicialización de registros cuánticos y clásicos
    qr = QuantumRegister(qubits_disponibles)
    cr = ClassicalRegister(qubits_disponibles)
    qc = QuantumCircuit(qr, cr)

    # Selección aleatoria de x0
    x0 = randint(1, N - 1)
    x_binario = np.zeros(qubits_disponibles, dtype=bool)

    # Conversión de N a binario y almacenamiento en x_binario
    for i in range(1, qubits_disponibles + 1):
        estado_bit = (N % (2 ** i) != 0)
        if estado_bit:
            N -= 2 ** (i - 1)
        x_binario[qubits_disponibles - i] = estado_bit

    # Aplicación de puertas X según el estado binario
    for i in range(0, qubits_disponibles):
        if x_binario[qubits_disponibles - i - 1]:
            qc.x(qr[i])
    x = x0

    # Bucle para encontrar el período 'r'
    while np.logical_or(x != x0, r <= 0):
        r += 1
        qc.measure(qr, cr)
        for i in range(0, 3):
            qc.x(qr[i])
        qc.cx(qr[2], qr[1])
        qc.cx(qr[1], qr[2])
        qc.cx(qr[2], qr[1])
        qc.cx(qr[1], qr[0])
        qc.cx(qr[0], qr[1])
        qc.cx(qr[1], qr[0])
        qc.cx(qr[3], qr[0])
        qc.cx(qr[0], qr[1])
        qc.cx(qr[1], qr[0])

        # Ejecución del trabajo en el simulador Aer
        trabajo = AerSimulator().run(qc, backend=simulador_qasm, shots=1024)
        resultado = trabajo.result()
        conteos = resultado.get_counts()

        print(qc)

        resultados = [[], []]
        for clave, valor in conteos.items():
            resultados[0].append(clave)
            resultados[1].append(int(valor))
        s = resultados[0][np.argmax(np.array(resultados[1]))]
    return r


In [12]:
# Función para romper la factorización usando el algoritmo de Shor
def romper_shor(N):
    N = int(N)
    while True:
        a = randint(0, N - 1)
        g = mcd(a, N)

        # Si el máximo común divisor es distinto de 1 o si N es igual a 1
        if g != 1 or N == 1:
            return g, N // g
        else:
            r = periodo(a, N)

            # Si el periodo no es par
            if r % 2 != 0:
                continue
            elif pow(a, r // 2, N) == -1:
                continue
            else:
                p = mcd(pow(a, r // 2) + 1, N)
                q = mcd(pow(a, r // 2) - 1, N)

                # Si p o q son iguales a N, continuar el bucle
                if p == N or q == N:
                    continue
                return p, q


In [10]:
# Función para encontrar el inverso modular de 'a' bajo el módulo 'm'
def inverso_modular(a, m):
    a = a % m  # Asegurarse de que 'a' esté en el rango de 0 a m-1
    for x in range(1, m):
        if (a * x) % m == 1:  # Encontrar x tal que (a * x) % m == 1
            return x
    return 1  # Si no se encuentra, devolver 1 como valor por defecto


In [13]:
# Asignar el valor de N de la clave pública
N_shor = publica[1]

# Asegurarse de que N_shor sea positivo
assert N_shor > 0, "La entrada debe ser positiva"

# Romper la factorización de N_shor utilizando el algoritmo de Shor
p, q = romper_shor(N_shor)

# Calcular phi (función totiente de Euler)
phi = (p - 1) * (q - 1)

# Calcular el inverso modular de la clave pública con respecto a phi
d_shor = inverso_modular(publica[0], phi)


Streaming output truncated to the last 5000 lines.
c28211: 16/══╩═══╩══╩══╩══╩══╩══╩══╩══╩══╩═══╩════╩════╩════╩════╩════╩════╩══»
             10  3  7  8  11 12 13 14 15 0   1    2    4    5    6    9    7  »
«                ┌───┐┌───┐     ┌───┐                          ┌─┐┌───┐     »
« q28212_0: ──■──┤ X ├┤ X ├──■──┤ X ├──────────────────────────┤M├┤ X ├─────»
«           ┌─┴─┐└─┬─┘└─┬─┘┌─┴─┐└─┬─┘                          └╥┘└┬─┬┘┌───┐»
« q28212_1: ┤ X ├──■────┼──┤ X ├──■─────────────────────────────╫──┤M├─┤ X ├»
«           └───┘       │  └───┘                       ┌─┐┌───┐ ║  └╥┘ └───┘»
« q28212_2: ────────────┼──────────────────────────────┤M├┤ X ├─╫───╫───────»
«                       │                              └╥┘└┬─┬┘ ║   ║       »
« q28212_3: ────────────■───────────────────────────────╫──┤M├──╫───╫───────»
«                                          ┌─┐          ║  └╥┘  ║   ║       »
« q28212_4: ───────────────────────────────┤M├──────────╫───╫───╫───╫───────»
«        

## Descifrar RSA mediante el algoritmo de Shor

In [14]:
# Desencriptar el mensaje utilizando el algoritmo de Shor
mensaje_desencriptado = desencriptar(objeto_encriptado, (d_shor, N_shor))

# Imprimir el mensaje desencriptado utilizando el algoritmo de Shor
print('\nMensaje descifrado usando el algoritmo de Shor: ' + mensaje_desencriptado + "\n")



Mensaje descifrado usando el algoritmo de Shor: fafas

